In [1]:
import pandas as pd

file_path = "Documents/r115_07tb2.txt"

# Step 1: Read all lines (including junk)
df = pd.read_csv(file_path,
                 sep=r'\s+',
                 header=None,
                 names=["INDEX", "YEAR", "MONTH", "DAY", "MAX", "MIN", "RAINFALL"],
                 skiprows=8, 
                 engine='python',
                 on_bad_lines='skip')

# Step 2: Remove lines with junk headers or dashes
df = df[~df["INDEX"].astype(str).str.contains("^-+|^INDEX", regex=True)]

# Step 3: Convert columns to appropriate numeric types
df[["INDEX", "YEAR", "MONTH", "DAY"]] = df[["INDEX", "YEAR", "MONTH", "DAY"]].apply(pd.to_numeric, errors='coerce')
df[["MAX", "MIN", "RAINFALL"]] = df[["MAX", "MIN", "RAINFALL"]].apply(pd.to_numeric, errors='coerce')

# Step 4: Drop rows missing date-related fields
df = df.dropna(subset=["INDEX", "YEAR", "MONTH", "DAY"])

# ✅ Step 5: Filter for station 42971 (Bhubaneswar)
df_42971 = df[df["INDEX"] == 42971].copy()

# Step 6: Reset index to start from 1
df_42971.reset_index(drop=True, inplace=True)
df_42971.index = df_42971.index + 1

# Show the cleaned DataFrame
df_42971


,INDEX,YEAR,MONTH,DAY,MAX,MIN,RAINFALL
1,42971.0,1970.0,1.0,1,26.5,11.2,0.0
2,42971.0,1970.0,1.0,2,27.6,11.4,0.0
3,42971.0,1970.0,1.0,3,27.1,11.1,0.0
4,42971.0,1970.0,1.0,4,27.2,10.8,0.0
5,42971.0,1970.0,1.0,5,28.6,13.1,0.0
...,...,...,...,...,...,...,...
16586,42971.0,2019.0,9.0,26,28.2,22.5,19.4
16587,42971.0,2019.0,9.0,27,30.9,23.5,1.4
16588,42971.0,2019.0,9.0,28,30.5,24.0,0.0
16589,42971.0,2019.0,9.0,29,31.7,23.8,9.7


In [3]:
# Step 1: Create daily averages from available data
daily_avg = (
    df_42971.groupby(["MONTH", "DAY"])[["MAX", "MIN", "RAINFALL"]]
    .mean()
    .reset_index()
    .rename(columns={
        "MAX": "MAX_IMPUTED",
        "MIN": "MIN_IMPUTED",
        "RAINFALL": "RAINFALL_IMPUTED"
    })
)

# Step 2: Merge with full dataset
df_imputed = df_42971.merge(daily_avg, on=["MONTH", "DAY"], how="left")

# Step 3: Impute missing values
df_imputed["MAX"] = df_imputed["MAX"].fillna(df_imputed["MAX_IMPUTED"])
df_imputed["MIN"] = df_imputed["MIN"].fillna(df_imputed["MIN_IMPUTED"])
df_imputed["RAINFALL"] = df_imputed["RAINFALL"].fillna(df_imputed["RAINFALL_IMPUTED"])

# Step 4: Drop helper columns
df_imputed.drop(columns=["MAX_IMPUTED", "MIN_IMPUTED", "RAINFALL_IMPUTED"], inplace=True)

# Optional: Reset index
df_imputed.sort_values(["YEAR", "MONTH", "DAY"], inplace=True)
df_imputed.reset_index(drop=True, inplace=True)

# Optional: Check if any missing values remain
print(df_imputed.isna().sum())


INDEX       0
YEAR        0
MONTH       0
DAY         0
MAX         0
MIN         0
RAINFALL    0
dtype: int64


In [5]:
# Step 1: Create "YEAR_MONTH" column
df_imputed["YEAR_MONTH"] = (
    df_imputed["YEAR"].astype(int).astype(str) + "-" +
    df_imputed["MONTH"].astype(int).astype(str).str.zfill(2)
)

# Step 2: Group by YEAR_MONTH
monthly_agg = df_imputed.groupby("YEAR_MONTH").agg({
    "MAX": "mean",       # Average Max Temp
    "MIN": "mean",       # Average Min Temp
    "RAINFALL": "sum"    # Cumulative Rainfall
})

# Step 3: Round values
monthly_agg = monthly_agg.round(2)

# Step 4: Reset index to bring YEAR_MONTH as a column
monthly_agg = monthly_agg.reset_index()

# Step 5: Start index from 1
monthly_agg.index = monthly_agg.index + 1

# Display all rows
pd.set_option("display.max_rows", None)
monthly_agg


,YEAR_MONTH,MAX,MIN,RAINFALL
1,1970-01,28.21,15.40,0.00
2,1970-02,31.34,19.94,44.00
3,1970-03,33.91,22.64,52.50
4,1970-04,36.76,24.63,23.10
5,1970-05,37.34,26.21,95.70
6,1970-06,33.62,25.30,337.70
7,1970-07,31.70,24.77,299.52
8,1970-08,31.45,24.85,399.00
9,1970-09,31.76,24.96,148.16
10,1970-10,31.04,22.39,148.30


In [13]:
import pandas as pd

# === Step 1: Read the ICRISAT monthly data files for Bhubaneswar ===
max_df = pd.read_csv("Documents/ICRISAT-District Level Data-6.csv")
min_df = pd.read_csv("Documents/ICRISAT-District Level Data-7.csv")
rain_df = pd.read_csv("Documents/ICRISAT-District Level Data-8.csv")

# === Step 2: Merge MAX, MIN, and RAINFALL data ===
merged_temp = pd.merge(max_df, min_df, on=["Dist Code", "Year", "State Code", "State Name", "Dist Name"], how="inner")
merged_all = pd.merge(merged_temp, rain_df, on=["Dist Code", "Year", "State Code", "State Name", "Dist Name"], how="inner")

# === Step 3: Reshape the merged ICRISAT data to long format ===
def melt_icrisat(df, value_type):
    month_map = {
        "JANUARY": "01", "FEBRUARY": "02", "MARCH": "03", "APRIL": "04",
        "MAY": "05", "JUNE": "06", "JULY": "07", "AUGUST": "08",
        "SEPTEMBER": "09", "OCTOBER": "10", "NOVEMBER": "11", "DECEMBER": "12"
    }
    id_vars = ["Year"]
    value_vars = [col for col in df.columns if value_type in col]
    df_melted = df.melt(id_vars=id_vars, value_vars=value_vars, var_name="Month", value_name=value_type)
    df_melted["Month"] = df_melted["Month"].str.extract(r"(\w+)")
    df_melted["MONTH"] = df_melted["Month"].map(month_map)
    df_melted["YEAR_MONTH"] = df_melted["Year"].astype(str) + "-" + df_melted["MONTH"]
    return df_melted[["YEAR_MONTH", value_type]]

# === Step 4: Create long-form DataFrames ===
max_long = melt_icrisat(merged_all, "MAXIMUM")
min_long = melt_icrisat(merged_all, "MINIMUM")
rain_long = melt_icrisat(merged_all, "PERCIPITATION")

# === Step 5: Merge all three into one long DataFrame ===
icrisat_long = max_long.merge(min_long, on="YEAR_MONTH").merge(rain_long, on="YEAR_MONTH")
icrisat_long.rename(columns={
    "MAXIMUM": "MAX",
    "MINIMUM": "MIN",
    "PERCIPITATION": "RAINFALL"
}, inplace=True)

# Round values
icrisat_long = icrisat_long.round(2)

# === Step 6: Merge ICRISAT data with original IMD monthly data for Bhubaneswar (index 42971) ===
monthly_merged_42971 = pd.concat([monthly_agg, icrisat_long], ignore_index=True)
monthly_merged_42971.drop_duplicates(subset=["YEAR_MONTH"], keep="first", inplace=True)
monthly_merged_42971 = monthly_merged_42971.sort_values("YEAR_MONTH").reset_index(drop=True)
monthly_merged_42971.index = monthly_merged_42971.index + 1

# === Final Output ===
monthly_merged_42971



,YEAR_MONTH,MAX,MIN,RAINFALL
1,1970-01,28.21,15.40,0.00
2,1970-02,31.34,19.94,44.00
3,1970-03,33.91,22.64,52.50
4,1970-04,36.76,24.63,23.10
5,1970-05,37.34,26.21,95.70
6,1970-06,33.62,25.30,337.70
7,1970-07,31.70,24.77,299.52
8,1970-08,31.45,24.85,399.00
9,1970-09,31.76,24.96,148.16
10,1970-10,31.04,22.39,148.30


In [15]:
monthly_merged_42971.to_csv("monthly_merged_42971.csv")

In [17]:
import numpy as np
import pandas as pd
from pymannkendall import original_test 
from statsmodels.tsa.stattools import acf

# === Function to prewhiten a series if lag-1 autocorrelation is strong ===
def prewhiten(series):
    r1 = acf(series, nlags=1, fft=False)[1]
    if abs(r1) < 0.1:
        return series, r1
    series_pw = series[1:] - r1 * series[:-1].values
    return series_pw, r1

# === Function to run Mann-Kendall test with autocorrelation check ===
def mk_with_autocorr_check(series):
    series = series.dropna()
    if len(series) < 10:
        return None, np.nan, "Insufficient data"
    r1 = acf(series, nlags=1, fft=False)[1]
    threshold = 0.2
    if abs(r1) > threshold:
        series_pw, r1_pw = prewhiten(series)
        if len(series_pw) < 10:
            return None, r1, "Prewhitened series too short"
        result = original_test(series_pw)
        method = "Modified MK"
    else:
        result = original_test(series)
        method = "Original MK"
    return result, r1, method

# === Apply the MK test to each month for all 3 variables ===
def apply_mk_test_with_autocorr(group):
    data = group[['MAX', 'MIN', 'RAINFALL']]
    results = []
    month_num = group.name
    for var in data.columns:
        series = data[var]
        result, r1, method = mk_with_autocorr_check(series)
        if result is None:
            trend = "Insufficient data"
            p_value = np.nan
            slope = np.nan
        else:
            trend = result.trend
            p_value = result.p
            slope = result.slope
        
        results.append({
            'Month': month_num,
            'Month_Name': pd.Timestamp(month=month_num, day=1, year=2000).strftime('%B'),
            'Variable': var,
            'Trend': trend,
            'p_value': p_value,
            'Sen_slope': slope,
            'Autocorr_lag1': r1,
            'Method_used': method
        })
    return pd.DataFrame(results)

# === Bhubaneswar-specific data processing ===
# Extract month number from YEAR_MONTH
monthly_merged_42971['Month'] = monthly_merged_42971['YEAR_MONTH'].str.split('-').str[1].astype(int)

# Apply Mann-Kendall test with autocorrelation correction
results_df_42971 = monthly_merged_42971.groupby('Month').apply(apply_mk_test_with_autocorr).reset_index(drop=True)

# Display result
print(results_df_42971)


    Month Month_Name  Variable       Trend   p_value  Sen_slope  \
0       1    January       MAX    no trend  0.140454   0.020000   
1       1    January       MIN    no trend  0.993122  -0.000185   
2       1    January  RAINFALL    no trend  0.526193   0.000000   
3       2   February       MAX  increasing  0.002146   0.055314   
4       2   February       MIN    no trend  0.903936  -0.001905   
5       2   February  RAINFALL    no trend  0.054812  -0.150000   
6       3      March       MAX  increasing  0.036191   0.027805   
7       3      March       MIN  increasing  0.033220   0.017913   
8       3      March  RAINFALL    no trend  0.260814  -0.053939   
9       4      April       MAX    no trend  0.190074   0.016667   
10      4      April       MIN    no trend  0.078638   0.013876   
11      4      April  RAINFALL    no trend  0.890291   0.020345   
12      5        May       MAX    no trend  0.742244   0.006548   
13      5        May       MIN  increasing  0.030745   0.01666

/var/folders/0_/t2qqj6n16kvbg3w2r6n9xnnm0000gn/T/ipykernel_46170/4276132508.py:66: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  results_df_42971 = monthly_merged_42971.groupby('Month').apply(apply_mk_test_with_autocorr).reset_index(drop=True)


In [19]:
def format_significance(p):
    if pd.isna(p):
        return 'NS'
    if p < 0.001:
        return '< 0.001'
    elif p < 0.01:
        return '1%'
    elif p < 0.05:
        return '5%'
    elif p < 0.10:
        return '10%'
    else:
        return 'NS'

def summarize_mk_results(results_df, var):
    # Filter only rows for this variable
    df_var = results_df[results_df['Variable'] == var].copy()
    
    # Prepare dataframe with desired columns and renaming
    summary = pd.DataFrame({
        'Month_Name': df_var['Month_Name'],
        f'{var}_MKz': df_var['Sen_slope'] / (df_var['Sen_slope'].std() if df_var['Sen_slope'].std() != 0 else 1),  # rough z-value proxy
        f'{var}_sig': df_var['p_value'].apply(format_significance),
        f'{var}_slope': df_var['Sen_slope'].round(4),
        f'{var}_RC': df_var['Autocorr_lag1'].round(2)
    })
    
    # Sort by Month number ascending (January to December)
    month_order = pd.to_datetime(summary['Month_Name'], format='%B').dt.month
    summary['Month_Order'] = month_order
    summary = summary.sort_values('Month_Order').drop(columns=['Month_Order']).reset_index(drop=True)
    
    return summary

# Create separate tables using Bhubaneswar MK results
max_summary_42971 = summarize_mk_results(results_df_42971, 'MAX')
min_summary_42971 = summarize_mk_results(results_df_42971, 'MIN')
rainfall_summary_42971 = summarize_mk_results(results_df_42971, 'RAINFALL')

# Print the summaries
print('--- MAX Trend (Bhubaneswar) ---')
print(max_summary_42971.to_string(index=False))

print('\n--- MIN Trend (Bhubaneswar) ---')
print(min_summary_42971.to_string(index=False))

print('\n--- RAINFALL Trend (Bhubaneswar) ---')
print(rainfall_summary_42971.to_string(index=False))


--- MAX Trend (Bhubaneswar) ---
Month_Name   MAX_MKz MAX_sig  MAX_slope  MAX_RC
   January  1.345326      NS     0.0200   -0.02
  February  3.720751      1%     0.0553    0.17
     March  1.870307      5%     0.0278   -0.19
     April  1.121105      NS     0.0167   -0.19
       May  0.440434      NS     0.0065   -0.08
      June  0.250927      NS     0.0037    0.11
      July  0.457066      NS     0.0068   -0.17
    August  1.394546      1%     0.0207    0.00
 September  0.771606      NS     0.0115   -0.03
   October -0.208205      NS    -0.0031    0.07
  November  1.109894      NS     0.0165    0.11
  December  1.189601      NS     0.0177   -0.25

--- MIN Trend (Bhubaneswar) ---
Month_Name   MIN_MKz MIN_sig  MIN_slope  MIN_RC
   January -0.025509      NS    -0.0002   -0.06
  February -0.262379      NS    -0.0019   -0.11
     March  2.467501      5%     0.0179   -0.02
     April  1.911410     10%     0.0139    0.15
       May  2.295814      5%     0.0167    0.05
      June  2.452912   